In [ ]:
import pandas as pd
import numpy as np
import keras

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Lambda, BatchNormalization, Flatten
from keras.layers import Dense, Dropout, Input
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.models import Model
from keras import layers

In [ ]:
X_train = np.load('../data/X_train.npy').reshape(36000,28,28,1)
X_valid = np.load('../data/X_valid.npy').reshape(6000,28,28,1)
X_score = np.load('../data/X_score.npy').reshape(28000,28,28,1)
y_train = np.load('../data/y_train.npy')
y_valid = np.load('../data/y_valid.npy')
y_score = pd.read_csv('../data/submit_v3.csv')
y_score = np.array(pd.get_dummies(y_score['Label']))
print(X_train.shape, y_train.shape)
print(X_score.shape, y_score.shape)
print(X_valid.shape, y_valid.shape)

In [ ]:
transform = {}
transform['width_shift_range'] = 0.075
transform['height_shift_range'] = 0.075
transform['rotation_range'] = 5
transform['shear_range'] = 0.3
transform['zoom_range'] = 0.075

generator = ImageDataGenerator(**transform)
generator.fit(X_train)

In [ ]:
def residual_block(inputs, filters):
    layer = Conv2D(filters,(3,3), strides=(1,1), padding='same', activation='relu')(inputs)
    layer = BatchNormalization(axis=-1)(layer)
    layer = Conv2D(filters,(3,3), strides=(1,1), padding='same', activation='relu')(layer)
    layer = BatchNormalization(axis=-1)(layer)
    layer = layers.add([layer, inputs])
    return layer

In [ ]:
inputs = Input(shape=(28,28,1))
layer = BatchNormalization(axis=-1)(inputs)
layer = Conv2D(32, (3,3), strides=(1,1), padding='same', activation='relu')(layer)
layer = BatchNormalization(axis=-1)(layer)
layer = Conv2D(32, (3,3), strides=(1,1), padding='same', activation='relu')(layer)
layer = BatchNormalization(axis=-1)(layer)
layer = MaxPooling2D(pool_size=(2,2))(layer)
layer = residual_block(inputs, 64)
layer = GlobalAveragePooling2D()(layer)
layer = BatchNormalization()(layer)
layer = Dense(512, activation='relu')(layer)
layer = Dropout(0.5)(layer)
output = Dense(10, activation='softmax')(layer)

In [ ]:
model = Model(inputs=inputs, outputs=output)

In [ ]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# model.summary()

In [ ]:
params = {}
params['epochs'] = 1
params['steps_per_epoch'] = 1200 # 36000 / 30
params['validation_data'] = (X_valid, y_valid)

In [ ]:
model.optimizer.lr = 0.001
params['epochs'] = 2
model.fit_generator(generator.flow(X_train, y_train, batch_size=30), **params)

In [ ]:
model.optimizer.lr = 0.01
params['epochs'] = 3
model.fit_generator(generator.flow(X_train, y_train, batch_size=30), **params)